In [1]:
import numpy as np
import matplotlib.pyplot as plt
import math
from sklearn import datasets

In [2]:
DATASET_TYPE = "simple" # simple, complex

In [3]:
def generate_examples(dataset_type = "simple", n_samples = 100, n_features = 1, noise = 20):
    if dataset_type == "simple":
        X, y = datasets.make_regression(n_samples=n_samples, n_features=n_features, n_informative=1, noise=noise, random_state=37)
        fig = plt.figure(figsize=(4, 4))
        plt.xlabel("x (feature)")
        plt.ylabel("y (output)")
        plt.title("Synthetic data set")
        plt.scatter(X, y)
        plt.show()
        return X, y
    
    elif dataset_type == "complex":
        n_samples = 300
        x = np.linspace(-10, 10, n_samples) # coordinates
        noise_sample = np.random.normal(0,0.5,n_samples)
        sine_wave = x + np.sin(4*x) + noise_sample
        plt.plot(x, sine_wave, 'o')
        plt.show()
        return x, sine_wave
    else:
        raise ValueError("Unknown dataset type: " + dataset_type)

In [4]:
class LinearRegression(object):
    """
    Modelul de Regresie Liniara
    y = X @ w 
        - valoarea prezisă de model
    t ~ N(t|X @ w, var) 
        - valorile de antrenare (target) sunt fac parte dintr-o distributie normala in jurul mean-ului X@w, 
          la care se adauga zgomot dat de varianta `var`
    """

    def fit(self, X, t):
        """
        Antrenarea modelului in sensul celor mai mici patrate (least squares fitting)
        Parameterii
        ----------
        X : (N, D) np.ndarray
            variabilele de intrare
        t : (N,) np.ndarray
            variabilele tinta (target / ground truth)
        """
        # TODO codul vostru aici: calculati vectorul de ponderi w
        # print(X.shape)
        N, D = X.shape
        
        # ar trebui facut cu np.dot sau cu @
        self.w = np.linalg.pinv(X) @ t
        # print(self.w)
        # end 
        
        self.var = np.mean(np.square(X @ self.w - t))

    def predict(self, X, return_std=False):
        """
        Intoarce valoari prezise de model pentru sample-uri X
        Parametrii
        ----------
        X : (N, D) np.ndarray
            sample-uri de valori D-dimensionale pentru care se doreste predictia valorii conform modelului
        return_std : bool, optional
            intoarce deviatia standard a fiecarei valori prezice, daca e setat pe True
        
        Intoarce
        -------
        y : (N,) np.ndarray
            vector de valori prezise
        y_std : (N,) np.ndarray
            deviatia standard a fiecarei valori prezise
        """
        # TODO codul vostru aici: calculati valoarea prezisa de modelul vostru
        N, D = X.shape
        y = np.zeros(N)

        y = X @ self.w
        # end codul vostru aici
        
        if return_std:
            # TODO codul vostru aici: intoarceti un vector de aceeasi dimensiune cu y, care 
            # are in fiecare pozitie o valoare egala cu deviatia standard a modelului antrenat (i.e. sqrt(var))
            y_std = np.ones_like(y) * math.sqrt(self.var)

            # aplic sqrt(var) pe toti y
            return y, y_std
        return y

In [ ]:
# Functia de eroare conform PDF
def mean_squared_error(y, t):
    return np.mean((y - t) * (y- t))